### UTS Data Mining - Crawling Twitter 

#### Analisis Kode - step by step.

Dalam proses data mining twitter, terdapat 3 proses.

1. Data Mining, menggunakan modul tweepy, akses token untuk autentifikasi, dan StreamListener() untuk streaming data
2. Pre-Processing, memasukkan data dan mengolahnya dengan Python menggunakan modul Pandas
3. Visualisasi, memberikan grafik visual dari data dengan menggunakan modul Matplotlib

#### Data Mining

Berikut adalah proses dari Data Mining, menggunakan:

1. modul tweepy (bisa didapatkan melalui conda install atau pip)

2. Akses token, akses token untuk API Twitter bisa didapatkan lewat website [Twitter Apps](https://apps.twitter.com/) dan mendaftarkan apps crawling untuk mendapatkan akses token. Akses token akan digunakan (embed) di dalam kode untuk autentifikasi saat streaming agar dapat berinteraksi dengan API Twitter

3. StreamListener() yang sudah termasuk dalam modul tweepy untuk melakukan streaming data. 

Berikut step untuk melakukan proses Data Mining

1. Import modul tweepy dan StreamListener()

2. Memasukkan akses token ke dalam variabel untuk autentifikasi yang nantinya akan disatukan dalam modul *OAuthHandler()*

3. Membuat class khusus untuk menyimpan data streaming dengan mengoper fungsi StreamListener() ke dalam class untuk melakukan proses read-write data twitter.

4. Modul tweepy dilengkapi dengan berbagai macam fungsi yang dapat digunakan untuk beberapa kebutuhan, salah satu yang digunakan dalam kode ini adalah filter() untuk melakukan penyaringan data twitter berdasarkan lokasi spesifik.

In [ ]:
from tweepy.streaming import StreamListener
from tweepy import OAuthHandler
from tweepy import Stream


# variable that contains the user credentials to access Twitter API
access_token = "" # Your Access Token
access_token_secret = "" # Your Access Token Secret
consumer_key = "" # Your Consumer Key
consumer_secret = "" # Your Consumer Secret


# this is the basic listener that just prints received tweets to stdout
class StdOutListener(StreamListener):
    def on_data(self, data):
        #print ("%s\n" % data)
        #return True
        with open("twitter_tweets.txt", "a") as tweet_log:
            tweet_log.write(data)
            
    def on_error(self, status):
        print (status)


l = StdOutListener()
auth = OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
stream = Stream(auth, l)


stream.filter(locations=[106.20, -6.41, 107.19, -6.01]) # Location set to Indonesia

Ketika kode di atas sudah dijalankan, proses read-write data twitter akan berlangsung di dalam file.txt yang sudah kita tentukan direktorinya sebelumnya (jika belum ditentukan maka akan berada di direktori script python).

Proses streaming yang dilakukan kali ini berjangka waktu 3 x 24 jam. Dimulai dari pukul 00.00 - 23.59 dan mengumpulkan 3 buat file txt berisi data twitter.

Setelah mendapatkan file txt dari streaming, program dapat diinterupsi (stop) untuk menghentikan proses streaming. Dengan itu berakhirlah proses data mining. Selanjutnya ke proses pre-processing data.

#### Pre-Processing

Proses pre-processing sebagian besar merupakan proses manipulasi *Data Frame* Python via modul Pandas. Setidaknya ada 2 modul yang perlu diimport, yaitu *json*, untuk melakukan load (read-write) data dalam format json dan membacanya menggunakan python; *pandas* untuk melakukan manipulasi data.

Berikut ini adalah proses read-write data ke dalam format json, dan langsung dimasukkan ke dalam *list* python. Berikut ada 3 kali proses read-write yang dilakukan. yang ditandai dengan
> for baris in file:
    >try:

Alasan mengapa dibuat 3 buah data dan tidak digabung menjadi 1 adalah karena analisa dari data nantinya akan 'terpisah' dan tidak disatukan dalam satu set list untuk memudahkan proses analisa.

Secara singkat alurnya:
1. Membuat list Python sesuai
2. Membuka file txt hasil streaming dengan mode 'read'
3. Melakukan iterasi ke setiap baris di dalam file txt dan mengubahnya menjadi json, lalu memasukkannya ke dalam list Python dengan *append()*
4. Try dan except berfungsi untuk mengabaikan jika terjadi error dalam pengambilan data.

In [1]:
import json

# membaca data dalam format json
tweets_data22Mar = []
tweets_file1 = open('twitter_tweets22Mar.txt', "r")
for line in tweets_file1:
    try:
        tweet = json.loads(line)
        tweets_data22Mar.append(tweet)
    except:
        continue

tweets_data1Apr = []
tweets_file2 = open('twitter_tweets1Apr.txt', "r")
for line in tweets_file2:
    try:
        tweet = json.loads(line)
        tweets_data1Apr.append(tweet)
    except:
        continue

tweets_data2Apr = []
tweets_file3 = open('twitter_tweets2Apr.txt', "r")
for line in tweets_file3:
    try:
        tweet = json.loads(line)
        tweets_data2Apr.append(tweet)
    except:
        continue

Setelah data masuk ke dalam list Python. Data diload ke dalam Data Frame Pandas. 

Dalam proses di bawah, dicontohkan pengambilan kategori 'text', 'lang', 'created_at' dan 'country' (secara berurutan: teks tweet, bahasa yang digunakan, kapan tweet dibuat, dan negara tempat tweet dikirimkan)

Penjelasan beberapa kode: 
- *lambda*  mengambil semua data yang ada dari sebuah kolom/baris. Dalam hal ini list.
- *map* memasukkan kategori pilihan ke dalam Data Frame
- *pd.DataFrame() membuat variabel bertipe Data Frame

In [2]:
import pandas as pd 

#loading data ke data frame

tweets22Mar = pd.DataFrame()
tweets22Mar["text"] = list(map(lambda tweet: tweet['text'], tweets_data22Mar))
tweets22Mar['lang'] = list(map(lambda tweet: tweet['lang'], tweets_data22Mar))
tweets22Mar['created_at'] = list(map(lambda tweet: tweet['created_at'], tweets_data22Mar))
tweets22Mar['country'] = list(map(lambda tweet: tweet['place']['country'] if tweet['place'] != None else None, tweets_data22Mar))

tweets1Apr = pd.DataFrame()
tweets1Apr["text"] = list(map(lambda tweet: tweet['text'], tweets_data1Apr))
tweets1Apr['lang'] = list(map(lambda tweet: tweet['lang'], tweets_data1Apr))
tweets1Apr['created_at'] = list(map(lambda tweet: tweet['created_at'], tweets_data1Apr))
tweets1Apr['country'] = list(map(lambda tweet: tweet['place']['country'] if tweet['place'] != None else None, tweets_data1Apr))

tweets2Apr = pd.DataFrame()
tweets2Apr["text"] = list(map(lambda tweet: tweet['text'], tweets_data2Apr))
tweets2Apr['lang'] = list(map(lambda tweet: tweet['lang'], tweets_data2Apr))
tweets2Apr['created_at'] = list(map(lambda tweet: tweet['created_at'], tweets_data2Apr))
tweets2Apr['country'] = list(map(lambda tweet: tweet['place']['country'] if tweet['place'] != None else None, tweets_data2Apr))

Setelah data berbentuk Data Frame, maka proses manipulasi data dapat dilakukan. Data Frame merupakan sebuah objek yang memiliki baris dan kolom, dalam proses memanipulasi data, baris dan kolom ini yang digunakan untuk mengakses dan melakukan sesuatu pada data.

Berikut dilakukan pengukuran terhadap panjang Data Frame dengan fungsi *len()* yang menggambarkan jumlah data di dalam Data Frame.

In [3]:
# print jumlah tweets
print ("Jumlah Data Tweets: ", len(tweets22Mar) + len(tweets1Apr) + len(tweets2Apr))
print(f"\nJumlah Data:\n22 Maret: {len(tweets22Mar)}\n1 April: {len(tweets1Apr)}\n2 April: {len(tweets2Apr)}\n")

Jumlah Data Tweets:  51455

Jumlah Data:
22 Maret: 15805
1 April: 18061
2 April: 17589



Berikut ini dilakukan akses terhadap kolom data 'lang' (bahasa) dan dilakukan proses filter data berbahasa Indonesia
> lang == 'in'

Dengan begitu hanya data dengan kategori bahasa Indonesia yang masuk ke variabel baru (dalam hal ini dilakukan proses overwrite variabel)

In [4]:
# filter data yang berbahasa Indonesia

tweetsIn22Mar = tweets22Mar[tweets22Mar.lang == 'in']
tweetsIn1Apr = tweets1Apr[tweets1Apr.lang == 'in'] 
tweetsIn2Apr = tweets2Apr[tweets2Apr.lang == 'in']

Proses overwrite sudah dilakukan dan selanjutnya dilakukan pengecekan terhadap Data Frame

**Notes** 
Di bawah ini adalah kode untuk mengaktifkan *InteractiveShell* Jupyter yang berfungsi agar proses pengecekan data dapat langsung sekaligus dilakukan dalam 1 sel kode.

In [5]:
# interaktif jupyter

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

Kode di bawah akan melakukan pengecekan pada data yang tadi kita ubah bahasanya. 

Untuk melakukan pengecekan dan melihat data dapat melalui beberapa cara, salah satu cara untuk mengecek adalah dengan *head* untuk melihat 5 kode teratas dari data sebagai perwakilan dari data yang lainnya. 

In [6]:
#cek print

tweetsIn22Mar.head()

tweetsIn1Apr.head()

tweetsIn2Apr.head()

,text,lang,created_at,country
4,W gabisa marah ama dia juga. Percuma marah ama...,in,Wed Mar 21 17:05:58 +0000 2018,Indonesia
5,mengaku sebagai istri di kehidupan yg sebelumn...,in,Wed Mar 21 17:06:01 +0000 2018,Indonesia
6,main game sambil tiduran 🤫 https://t.co/aSotVk...,in,Wed Mar 21 17:06:01 +0000 2018,Indonesia
8,Hari ini muka berasa bgt perih2 krn hbs di tus...,in,Wed Mar 21 17:06:24 +0000 2018,Indonesia
9,@anisaaurelias lah kok batre?,in,Wed Mar 21 17:06:34 +0000 2018,Indonesia


,text,lang,created_at,country
0,yg katanya besok pagi mau ikut race. tapi jam ...,in,Sat Mar 31 17:00:44 +0000 2018,Indonesia
1,@ikramarki @satriaoo Isinya bulu Ayam boiler y...,in,Sat Mar 31 17:00:46 +0000 2018,Indonesia
2,@an_zefa @detikcom Stju bro..wkwkwkw #anggurmerah,in,Sat Mar 31 17:00:47 +0000 2018,Indonesia
4,@SammiSoh Buat apa kita manusia tanggapin otak...,in,Sat Mar 31 17:01:01 +0000 2018,Indonesia
5,Bareng si cantik #BabyALUDRA .....duh hiburan ...,in,Sat Mar 31 17:01:05 +0000 2018,Indonesia


,text,lang,created_at,country
0,Salah satu yang enak banget yang bahasa Tagalo...,in,Sun Apr 01 16:12:31 +0000 2018,Indonesia
1,Closer jadi penutup konser @TheChainsmokers . ...,in,Sun Apr 01 16:12:34 +0000 2018,Indonesia
3,"Akhirnya Cikarang again,, sinyak siuman semua...",in,Sun Apr 01 16:12:40 +0000 2018,Indonesia
4,@NgopiJahe_09 Makanya lu jangan kebanyakan mak...,in,Sun Apr 01 16:12:41 +0000 2018,Indonesia
5,ISTIKHARAH CINTAKU\n\n￼… https://t.co/BdSvhVFCRa,in,Sun Apr 01 16:12:44 +0000 2018,Indonesia


Selanjutnya dilakukan pengecekan terhadap jumlah data,

In [7]:
# print jumlah tweets indo

print ("Jumlah Data Tweets Indonesia :", len(tweetsIn22Mar) + len(tweetsIn1Apr) + len(tweetsIn2Apr))
print(f"Jumlah Data:\n22 Maret: {len(tweetsIn22Mar)}\n1 April: {len(tweetsIn1Apr)}\n2 April: {len(tweetsIn2Apr)}\n")

Jumlah Data Tweets Indonesia : 36270
Jumlah Data:
22 Maret: 11363
1 April: 12301
2 April: 12606



Terlihat jumlah data sudah berkurang dari 51455 menjadi 36270

1 proses manipulasi data telah dilakukan. Proses manipulasi berikutnya adalah mengubah *index* Data Frame menjadi zona waktu (yang diwakili oleh kategori *created_at* yang berformat date/time)

In [8]:
# zona waktu as index

tweetsIn22Mar.index = pd.to_datetime(tweetsIn22Mar['created_at'], utc=True)
tweetsIn1Apr.index = pd.to_datetime(tweetsIn1Apr['created_at'], utc=True)
tweetsIn2Apr.index = pd.to_datetime(tweetsIn2Apr['created_at'], utc=True)

Index adalah salah satu fitur penting dari Data Frame. Secara singkat, Index adalah *unique key* (primary key dalam database) untuk mengakses data di sebuah baris. 

Ketika Index tidak ditentukan, maka akan otomatis bernilai 0 sampai n-1 data.

Saat ini manipulasi terhadap Index sudah dilakukan dan Index sebelumnya telah dioverwrite menjadi *created_at* yang mewakili tanggal/waktu. 

**Perlu diingat** bahwa tidak semua kategori dapat dijadikan Index. Seperti halnya *primary key* atau *unique key* maka setiap Index harus bersifat unik bagi setiap baris. Hal ini dikarenakan setiap baris memiliki data dan proses pengambilan data ke dalam baris hanya bisa dilakukan ketika baris satu dan yang lain memiliki 'address' yang berbeda. *Created_at* dapat dijadikan Index karena bersifat unik bagi setiap tweets yang ada (karena proses streaming dan bukan batching, jadi tidak mungkin ada tweet yang masuk dalam waktu bersamaan)

Berikut ini dilakukan pengecekan lagi terhadap Data Frame.

Dapat dilihat perbedaannya di kolom sebelah kiri menjadi *created_at*

In [ ]:
#cek print head - zona waktu as index

tweetsIn22Mar.head()
tweetsIn1Apr.head()
tweetsIn2Apr.head()

Sudah 2 manipulasi dilakukan. Penyaringan data, dan penentuan Index.

Selanjutnya akan dilakukan penyesuaian zona waktu, karena data yang didapatkan masih berzona waktu GMT 00.00, maka waktu akan diubah ke zona Asia/Jakarta dengan melakukan import modul *timezone* 

In [ ]:
# penyesuaian zona waktu @ Indonesia

import pytz
from datetime import datetime
from pytz import timezone

JKT = pytz.timezone('Asia/Jakarta')
tweetsIn22Mar.index = tweetsIn22Mar.index.tz_convert(JKT)
tweetsIn1Apr.index = tweetsIn1Apr.index.tz_convert(JKT)
tweetsIn2Apr.index = tweetsIn2Apr.index.tz_convert(JKT)

Lakukan pengecekan lagi terhadap Data Frame.

In [ ]:
# melihat hasil perubahan zona waktu di kolom index

tweetsIn22Mar.head()
tweetsIn1Apr.head()
tweetsIn2Apr.head()

Bisa dilihat nilai dari Index berubah menjadi waktu Indonesia.

Dengan ini sudah 3x manipulasi dilakukan. Selanjutnya akan dilakukan penyaringan data untuk waktu pas 1 hari dalam tanggal tertentu. Dalam contoh berikut, data akan diambil dari tanggal **22 Maret, 1 April, dan 2 April**

Di bawah ini, fungsi *loc()* berfungsi untuk *me-locate* (menemukan) data di dalam baris dan kolom Data Frame dengan value spesifik seperti yang tertera di dalam kurung siku.

In [ ]:
# indexer untuk tanggal pembuatan status - pas di tanggal tersebut

tweetsIn22Mar = tweetsIn22Mar.loc['2018-03-22']
tweetsIn1Apr = tweetsIn1Apr.loc['2018-04-1']
tweetsIn2Apr = tweetsIn2Apr.loc['2018-04-2']

Sudah 4x manipulasi dilakukan, selanjutnya dilakukan pengecekan terhadap *head()* dan juga *tail()*. Jika head merupakan 5 data teratas dalam Data Frame, maka tail adalah 5 data terakhir yang ada di dalam Data Frame.

Pengecekan pada awal dan akhir Data Frame memastikan bahwa tidak hanya bagian depan data yang berhasil diubah, dan jika bagian depan dan belakang berubah, dapat diasumsikan bahwa manipulasi data berjalan lancar dari awal ke akhir (artinya semua data dimanipulasi)

In [ ]:
# cek head & tail untuk perubahan tanggal

tweetsIn22Mar.head()
tweetsIn22Mar.tail()

tweetsIn1Apr.head()
tweetsIn1Apr.tail()

tweetsIn2Apr.head()
tweetsIn2Apr.tail()

Dapat dilihat terdapat perubahan di tanggal 2 April, dimana tweet yang berasal dari tanggal 1 April dikeluarkan. 

Melihat bahwa 5 data terdepan dan terbelakang dari Data Frame memiliki tanggal yang sama. Hal ini memastikan bahwa **Data representatif untuk mewakili tanggal tersebut**

Selanjutnya dilakukan perhitungan untuk mengecek jumlah data yang tersisa setelah filter.

**Dan**

menyelesaikan proses pre-processing dengan mengubah Data Frame ke bentuk yang lebih simpel untuk **memilih kategori** yang akan divisualisasikan. Dalam hal ini, yang akan dijadikan visualisasi adalah *jumlah tweets* dan *tanggal* yang mungkin memiliki hubungan di antaranya.

In [ ]:
print ("Jumlah data tweets berbahasa Indonesia berdasarkan tanggal\n")
print("Total Tweets: ", len(tweetsIn22Mar) + len(tweetsIn1Apr) + len(tweetsIn2Apr))

df22m = pd.DataFrame({'tanggal' : ['22 Maret 2018'], 'jumlah tweets' : [len(tweetsIn22Mar)]})
df1ap = pd.DataFrame({'tanggal' : ['1 April 2018'], 'jumlah tweets' : [len(tweetsIn1Apr)]})
df2ap = pd.DataFrame({'tanggal' : ['2 April 2018'], 'jumlah tweets' : [len(tweetsIn2Apr)]})

# Print Jumlah Tweets
df22m.head()  # Mar 22
df1ap.head()   # Apr 1
df2ap.head()   # Apr 2

Dapat dilihat, data yang tadinya 36270 entry menjadi 35684 entry. Dan terjadi perubahan terhadap Data Frame yang sekarang hanya berupa 1 entry data yang berisi *jumlah tweets* dan *tanggal*

Sebelumnya perlu diperhatikan bahwa tidak dilakukan *overwrite* variabel dalam proses di atas. Artinya Data Frame yang sebelumnya sudah melewati proses manipulasi masih bisa diakses dan digunakan. Karena itulah digunakan variabel baru (Data Frame baru) untuk dimasukkan ke proses visualisasi.

#### Visualisasi

Sekarang akan dilakukan proses visualisasi dari data. 

Pada proses ini akan digunakan beberapa modul khusus visualisasi, seperti matplotlib, seaborn, dan pyplot.

Ketiga modul tersebut adalah modul-modul visualisasi yang mendukung proses visualisasi data dan dalam langkah-langkah selanjutnya akan digunakan beberapa (banyak) fungsi dari modul-modul tersebut (kecuali seaborn dan pyplot, mayoritas kode ini menggunakan matplotlib untuk visualisasi)

Berikut diimport modul matplotlib untuk visualisasi (dan numpy, hanya untuk melakukan iterasi array)

Sebelum melihat alur kode, mungkin bisa dilihat terlebih dahulu output dari kode yang dihasilkan. Kali ini visualisasi akan menggunakan *bar* untuk mengukur jumlah tweets perharinya.

Alur kode visualisasi:
1. Membuat objek kanvas dari fig (objek figure) bernama 'ax'. Objek ax lalu diberikan fungsi subplots dari matplotlib sebagai tools untuk 'mewarnai' kanvas tersebut
2. Memasukkan mengubah jumlah tweets menjadi string untuk digunakan sebagai *label* dalam *legends*
3. Menetapkan lebar dari bar yang akan digunakan, sebesar 0.3 inch

4. Membuat variabel rects yang merupakan representasi dari bar di dalam kanvas dan menggambarkan bar dengan tools *bar()* dari matplotlib yang dapat diakses melalui objek ax.bar. Beberapa argumen dimasukkan ke dalam fungsi ax.bar (fungsi pembuat bar), secara berurutan *index (pemberi nomor bar), height (item yang diukur), lebar bar, warna bar, dan label untuk legend*

5. Selanjutnya objek ax yang berupa kanvas akan 'dihiasi' dengan informasi seperti label, judul, xticks (penanda data), dan legend.

In [ ]:
# melihat grafik bar total tweets per hari

import matplotlib.pyplot as plt
import numpy as np

fig, ax = plt.subplots()

mar22 = str(df22m.loc[0, 'jumlah tweets'])
apr1 = str(df1ap.loc[0, 'jumlah tweets'])
apr2 = str(df2ap.loc[0, 'jumlah tweets'])

bar_width = 0.3

rects1 = ax.bar(1, df22m['jumlah tweets'], bar_width, color = 'b', label = mar22)
rects2 = ax.bar(2, df1ap['jumlah tweets'], bar_width, color = 'r', label = apr1)
rects3 = ax.bar(3, df2ap['jumlah tweets'], bar_width, color = 'y', label = apr2)

ax.set_ylabel('Jumlah Tweets')
ax.set_xlabel('Tanggal')
ax.set_title('Total Tweets per Hari')
ax.set_xticks(np.arange(1, 4, step = 1))
ax.set_xticklabels(('22 Maret', '1 April', '2 April'))
ax.legend(loc = 0)

Dapat dilihat hasil visualisasi dari total tweets dan tanggal (yang belum dinamis)

Proses selanjutnya sama seperti proses sebelumnya, berupa pengolahan data untuk divisualisasikan.

Di bawah ini beberapa modul diimport.

**Dan salah satu yang penting adalah command *% matplotlib inline* **

Command %matplotlib inline, yang sering dinamakan magic command matplotlib, berguna untuk menghasilkan visualisasi (gambar; figure) agar dapat dilihat di dalam Jupyer. 

Jika kode sebelumnya dieksekusi tanpa magic command dan membuka figure di window baru, maka dengan mengaktifkan magic command ini, visualisasi dapat dilakukan di dalam Jupyter.

In [ ]:
# membuat tabel dan grafik data yang berisi jumlah total tweets per jam pada tanggal tertentu

import matplotlib.pyplot as plt
import seaborn as sns
from pylab import *
%matplotlib inline

plt.rcParams['figure.figsize'] = (15, 5)

Di bawah ini (3 sel di bawah ini) bukanlah kode yang merupakan visualisasi dari matplotlib. Tapi dilakukan dari Data Frame Pandas.

Matplotlib memang terlihat *fancy* untuk visualisasi data, namun visualisasi tidak hanya lewat matplotlib; pandas, seaborn, plotly dan geographical plotting adalah modul lainnya yang bisa digunakan untuk visualisasi.

Kali ini akan digunakan Pandas untuk visualisasi jumlah tweet yang dihasilkan setiap jamnya, dan akan divisualisasikan melalui matplotlib dalam bentuk *bar* dan *line*.

In [ ]:
# membuat tabel dan grafik data jumlah tweets

from pandas import Series
def f(x):
     return Series(dict(Number_of_tweets = x['text'].count()))
    
hourly_count22m = tweetsIn22Mar.groupby(tweetsIn22Mar.index.hour).apply(f)
print ("data selama %d jam pada tanggal 22 Maret 2018" % len(hourly_count22m))
hourly_count22m

Sebelumnya sudah diperagakan beberapa fungsi dari Data Frame pandas. Sekarang akan digunakan fungsi *group by* yang mengelompokkan data sesuai argumen yang diberikan, dalam hal ini jam *hour*.

Alur kode:

1. Melakukan import Series. Series merupakan bentuk 1D yang sifatnya sama dengan Data Frame, dan hanya berbeda dimensi)
2. Membentuk fungsi yang mengoper x (dalam hal ini sebuah variabel hasil groupby dari hour tweets)
3. Menempatkan hasil groupby ke dalam variabel baru hourly_count
4. Melakukan print hourly_count

**Dua baris sel di bawah juga melakukan hal yang sama**

In [ ]:
hourly_count1ap = tweetsIn1Apr.groupby(tweetsIn1Apr.index.hour).apply(f)
print ("data selama %d jam pada tanggal 1 April 2018" % len(hourly_count1ap))
hourly_count1ap

In [ ]:
hourly_count2ap = tweetsIn2Apr.groupby(tweetsIn2Apr.index.hour).apply(f)
print ("data selama %d jam pada tanggal 2 April 2018" % len(hourly_count2ap))
hourly_count2ap

Dari visualisasi di atas dapat dilihat jumlah spesifik data pertanggal, perjamnya.

Selanjutnya akan dilakukan visualisasi dengan bar dan lines melalui matplotlib.

Alur kode visualisaasi:

1. Memberikan tools plt kepada variabel fig
2. Membuat objek kanvas bernama 'ax' dari fig.
3. Membuat plotting lines dengan hourly_count masing-masing tanggal yang ditandai *label* dengan mengoper kolom 'Number_of_tweets'
4. Melakukan kustomisasi kanvas berupa title, label, xticks dan legend.

In [ ]:
fig = plt.figure()

ax = fig.add_axes([0,0,0.8,0.8])

ax.plot(hourly_count22m['Number_of_tweets'], label = '22 Maret')
ax.plot(hourly_count1ap['Number_of_tweets'], label = '1 April')
ax.plot(hourly_count2ap['Number_of_tweets'], label = '2 April')

ax.set_title('Total Tweets per Jam', fontsize=15)
ax.set_xlabel('Jam', weight='bold', labelpad=15)
ax.set_ylabel('Jumlah Tweets', weight='bold', labelpad=15)

hours = list(range(0,24))
xticks(np.arange(24), hours, rotation = 0,fontsize = 9)

ax.legend()

Kurang lebih sama seperti visualisasi sebelumnya. Hanya berbeda kategori yang divisualisasikan dengan cara yang berbeda (lines)

Selanjutnya pun kurang lebih sama, dengan menggunakan cara bar.

In [ ]:
fig, ax = plt.subplots()

#hours = list(range(0,24))
hours = np.arange(0, 24)
bar_width = 0.2

ax.bar(hours - bar_width, hourly_count22m['Number_of_tweets'], bar_width, label = '22 Maret')
ax.bar(hours, hourly_count1ap['Number_of_tweets'], bar_width, label = '1 April')
ax.bar(hours + bar_width, hourly_count2ap['Number_of_tweets'], bar_width, label = '2 April')

ax.set_title('Total Tweets per Jam', fontsize=15)
ax.set_xlabel('Jam', weight='bold', labelpad=15)
ax.set_ylabel('Jumlah Tweets', weight='bold', labelpad=15)

xticks(np.arange(24), hours, rotation = 0,fontsize = 9)
ax.set_xticks(hours + bar_width / 2)
#ax.set_xticklabels(np.arange(24), hours, rotation = 0,fontsize = 9)
ax.legend()

ax.legend()

#### Penutup

Dengan selesainya tahapan dari data mining twitter ini, akan diberikan kesimpulan:

1. Proses yang dilakukan dalam aktivitas kali ini terdiri dari **Data Mining** atau proses pengambilan data, **Pre-Processing** atau proses manipulasi data, **Visualisasi** atau proses penggambaran dari data

2. Library yang dibutuhkan diantaranya *tweepy, json, pandas, matplotlib* untuk melakukan proses di atas.

3. Tweepy merupakan library yang memungkinkan script python berinteraksi dan mengambil data dari API Twitter

4. Untuk melakukan loading (read-write) data digunaan library json

5. Pandas; Data Frame dan Seriesnya adalah tools untuk memanipulasi data dengan banyak fungsi yang menyertainya. Manipulasi data di python selalu menggunakan Pandas

6. Matplotlib adalah salah satu library untuk melakukan visualisasi. Di antara cara visualisasi lainnya dapat menggunakan Data Frame pandas, seaborn, pyplot dan geographical plotting.